# 1. 기본 프롬프트

In [2]:
from langchain.llms import OpenAI
import os
import config as cfg

os.environ['OPENAI_API_KEY'] = cfg.gpt_config["api_key"]

llm = OpenAI(model_name='text-davinci-003', temperature=0.9)
llm('1980년대 메탈 음악 5곡 추천해줘.')

'\n\n1. 미투데이 - 드래곤타운\n2. 기가막힌 - 레인보우\n3. 소나타 - 스코틀랜드\n4. 헤이즈 - 신\n5. 바티칸 다크니스 - 메탈리카'

# Load chroma DB

In [10]:
import chromadb
client = chromadb.PersistentClient(path="chromadb")
    
collection1 = client.get_or_create_collection(
    name="law_table",
    metadata={"hnsw:space": "cosine"}
)

collection2 = client.get_or_create_collection(
    name="detail_law_table",
    metadata={"hnsw:space": "cosine"}
)

query_results_1 = collection1.query(
    query_texts=["인권"],
    n_results=5,
    where={"카테고리":"인권"},
)
    
query_results_2 = collection2.query(
    query_texts=["인권"],
    n_results=5,
    where={"카테고리":"인권"},
)

   
documents_list1 = [doc for sublist in query_results_1['documents'] for doc in sublist]
documents_list2 = [doc for sublist in query_results_2['documents'] for doc in sublist]
    
list1 = str(documents_list1)
list2 = str(documents_list2)

# GPT 3-5

In [3]:
from langchain.chat_models import ChatOpenAI
from langchain.schema import (
    AIMessage,
    HumanMessage,
    SystemMessage
)

chat = ChatOpenAI(model_name='gpt-3.5-turbo', temperature=0.9)
sys = SystemMessage(content="당신은 음악 추천을 해주는 전문 AI입니다.")
msg = HumanMessage(content='1980년대 메탈 음악 5곡 추천해줘.')

aimsg = chat([sys, msg])
aimsg.content

'1980년대의 메탈 음악은 많은 전설적인 곡들이 있습니다. 여기 몇 가지 추천해 드리겠습니다:\n\n1. Iron Maiden - "The Trooper"\n이 말이 필요한 부드럽고 강렬한 리프는 거침없는 터키쉬 마르카 리듬과 어우러져 매력적인 곡입니다.\n\n2. Metallica - "Master of Puppets"\n이 곡은 메탈의 거장 Metallica의 신화적인 앨범 \'Master of Puppets\'의 타이틀 트랙입니다. 힘찬 기타 리프, 잔인한 드럼과 단단한 노래 가사로 구성된 이 곡은 메탈 역사상 가장 아이코닉한 곡 중 하나입니다.\n\n3. Judas Priest - "Breaking the Law"\n80년대의 메탈 음악을 상징하는 곡으로, Judas Priest의 대표작 중 하나입니다. 중독성 있는 리프와 강력한 가사로 유명한 이 곡은 메탈의 상징적인 애덤스 애플을 거쳐 나온 작품입니다.\n\n4. Black Sabbath - "Heaven and Hell"\nBlack Sabbath의 목소리로 유명한 Ronnie James Dio가 합류한 후의 작품으로, 이 곡은 그들의 대표작 중 하나입니다. 그의 강렬한 보컬과 훌륭한 기타 리프로 인해 이 곡은 메탈 팬들 사이에서 매우 사랑받고 있습니다.\n\n5. Megadeth - "Peace Sells"\n메탈릭한 리프와 진중한 가사로 유명한 이 곡은 Megadeth의 대표작 중 하나입니다. 풍부한 악기 연주와 Dave Mustaine의 고독하면서도 독특한 보컬로, 이 곡은 80년대 메탈 음악의 명곡 중 하나입니다.\n\n이것들은 1980년대 메탈 음악의 일부에 불과하며, 여전히 수많은 훌륭한 곡들이 있습니다. 메탈을 좋아한다면, 이런 곡들을 듣고 그 시대의 명곡들을 더 자세히 탐색해보시길 추천드립니다.'

# 2. 프롬프트 템플릿

In [11]:
from langchain.prompts import PromptTemplate

'''
prompt = PromptTemplate(
    input_variables=["상품"],
    template="{상품} 만드는 회사 이름 추천해줘. 기억에 남는 한글 이름으로"
)

prompt.format(상품="AI 여행 추천서비스")
'''
prompt = PromptTemplate(
    input_variables=["상품"],
    template="{기관}의 최신 안건을 정리해줘"
)

prompt.format(기관="국회")

'국회의 최신 안건을 정리해줘'

In [12]:
from langchain.chains import LLMChain

chain = LLMChain(llm=chat, prompt=prompt)

chain.run(기관=list1)

'- 장애인권리보장법안(대안)(보건복지위원장)\n   - 의안 일자: 2023-11-23\n   - 의안번호: DD11479\n   - 단계: 접수\n   - 세부단계: 제안\n   - 소관위원회: -\n   \n- 희귀질환관리법 일부개정법률안(대안)(보건복지위원장)\n   - 의안 일자: 2023-11-23\n   - 의안번호: DD11482\n   - 단계: 접수\n   - 세부단계: 제안\n   - 소관위원회: -\n   \n- 주민등록법 일부개정법률안(대안)(행정안전위원장)\n   - 의안 일자: 2023-11-23\n   - 의안번호: DD11488\n   - 단계: 접수\n   - 세부단계: 제안\n   - 소관위원회: -\n\n- 병역법 일부개정법률안(대안)(국방위원장)\n   - 의안 일자: 2023-11-23\n   - 의안번호: DD11476\n   - 단계: 접수\n   - 세부단계: 제안\n   - 소관위원회: -\n   \n- 장애인차별금지 및 권리구제 등에 관한 법률 일부개정법률안(노웅래의원 등 10인)\n   - 의안 일자: 2023-11-22\n   - 의안번호: 2125551\n   - 단계: 위원회 심사\n   - 세부단계: 회부\n   - 소관위원회: 보건복지위원회'